In [26]:
import heapq
import datetime
import collections
import itertools


# Part I - The Node Class

## Task 1

In [2]:
class Node:
    
    def __init__(self,name):
        self._name = name
        self._neighbors = {}
    
    def __str__(self):
        msg = "Node: {}\nEdges: {}\n".format(self._name,len(self))
        msg += ''.join("  Edge: ({},{}) Weight: {}\n".format(self._name,n,w) for (n,w) in self._neighbors.items())
        return msg
    
    def __len__(self):
        return len(self._neighbors)
    
    def __contains__(self,item):
        if isinstance(item, Node):
            item = item._name
        return item in self._neighbors
    
    def __getitem__(self, key):
        if isinstance(key, Node):
            key = key._name
        return self._neighbors.get(key, None)
    
    def __setitem__(self, key, item):
        if isinstance(key,Node):
            key = key._name
        if self._name != key:
            self._neighbors[key] = max(self._neighbors.get(key,item),item)
    
    def __eq__(self, other):
        return self._name == other._name
    
    def __ne__(self, other):
        return self._name != other._name
     
    def is_neighbor(self, name):
        return name in self
    
    def update(self, name, weight):
        self[name] = weight
        
    def update(self, other):
        if self == other:
            for neighbor, weight in other._neighbors.items():
                self[neighbor] = weight
            
    def remove_neighbor(self, key):
        if key in self:
            if isinstance(name, Node):
                key = key._name
            del self._neighbors[key]
    
    def is_isolated(self):
        return not self._neighbors
    
    def name(self):
        return self._name
    
    def edges(self):
        return self._neighbors.items()
    



## Task 2 - Exemplary Usage

### Question 1

In [3]:
nodes = [Node(n) for n in range(1,11)]
edges = [[2,4,5,6,7],[3,4],[2,4],[5],[6],[],[6],[1,2,7],[2,8,10],[2,3]]
weights = [[10,20,20,5,15],[5,10],[15,5],[10],[5],[],[10],[5,20,5],[15,20,10],[5,15]]
for n,es,ws in zip(nodes,edges,weights):
    for e,w in zip(es,ws):
        n[e]=w
    print(n)

Node: 1
Edges: 5
  Edge: (1,2) Weight: 10
  Edge: (1,4) Weight: 20
  Edge: (1,5) Weight: 20
  Edge: (1,6) Weight: 5
  Edge: (1,7) Weight: 15

Node: 2
Edges: 2
  Edge: (2,3) Weight: 5
  Edge: (2,4) Weight: 10

Node: 3
Edges: 2
  Edge: (3,2) Weight: 15
  Edge: (3,4) Weight: 5

Node: 4
Edges: 1
  Edge: (4,5) Weight: 10

Node: 5
Edges: 1
  Edge: (5,6) Weight: 5

Node: 6
Edges: 0

Node: 7
Edges: 1
  Edge: (7,6) Weight: 10

Node: 8
Edges: 3
  Edge: (8,1) Weight: 5
  Edge: (8,2) Weight: 20
  Edge: (8,7) Weight: 5

Node: 9
Edges: 3
  Edge: (9,2) Weight: 15
  Edge: (9,8) Weight: 20
  Edge: (9,10) Weight: 10

Node: 10
Edges: 2
  Edge: (10,2) Weight: 5
  Edge: (10,3) Weight: 15



### Question2

### Question 3

In [4]:
print("Total number of edges: {}".format(sum(len(n) for n in nodes)))
print("Overal weight: {}".format(sum(w for (e,w) in n.edges() for n in nodes)))

Total number of edges: 20
Overal weight: 200


### Question 4

In [5]:
nodes.sort(key=len)
print([n.name() for n in nodes])

[6, 4, 5, 7, 2, 3, 10, 8, 9, 1]


# Part II - The Graph Class

## Task 1

In [6]:
class Graph(object):
    
    def __init__(self, name, nodes=[]):
        self._name = name
        self._nodes = {}
        self._spt = {}   #shortest path tree.
        for node in nodes:
            self.update(node)
    
    def __str__(self):
        msg="Graph: {}\nNodes: {}\n".format(self._name, len(self))
        msg += ''.join("  Name: {} Neighbors: {}\n".format(node,len(list(self.neighbors(node)))) for node in self._nodes)
        edges = ''.join("  Edge: ({},{}) Weight: {}\n".format(frm_n,to_n,w) for frm_n in self._nodes \
                        for (to_n,w) in self.edges_from(frm_n))
        msg += "Edges: {}\n".format(edges.count("Edge: "))
        msg += edges
        return msg
    
    def __len__(self):
        return len(self._nodes)
    
    def __contains__(self,key):
        if isinstance(key,Node):
            key = key._name
        return key in self._nodes

    def __getitem__(self,name):
        return self._nodes[name]
    
    def __add__(self,other):
        new_graph = Graph(self._name + "_" + other._name,[])
        new_graph._nodes = self._nodes.copy()
        for node in other._nodes.values():
            new_graph.update(node)
        return new_graph
    
    def __iter__(self):
        return (node for node in self._nodes)
    
    def update(self,node):
        if isinstance(node,Node):
            if node in self:
                self[node.name()].update(node)
            else:
                self._nodes[node.name()] = node
            if self._spt:
                self._spt.clear()
        
    def edges_from(self, frm_name):
        return ((to_name,weight) for (to_name, weight) in self[frm_name].edges() if to_name in self)
    
    def neighbors(self, of_name):
        return(node for node in self._nodes if node in self[of_name])
        
    def remove_node(self, name):
        if name in self:
            del self._nodes[name]
            if self._spt:
                self._spt.clear()
    
    def is_edge(self, frm_name, to_name):
        return frm_name in self and to_name in self and to_name in self[frm_name]
    
    def add_edge(self, frm_name, to_name, weigth):
        if frm_name in self and to_name in self:
            self[frm_name][to_name] = weigth
            if self._spt:
                self._spt.clear()
    
    def build_edge(self, frm_name, to_name, weigth):
        frm_node = Node(frm_name)
        to_node = Node(to_name)
        frm_node[to_node]= weigth
        self.update(frm_node)
        self.update(to_node)
    
    def remove_edge(self,frm_name, to_name):
        if frm_name in self and to_name in self:
            self[frm_name].remove_neighbor(to_name)
            if self._spt:
                self._spt.clear()
    
    def get_edge_weight(self,frm_name, to_name):
        return_value = None
        if frm_name in self and to_name in self:
            return_value = self[frm_name][to_name]
        return return_value
    
    def get_path_weight(self, path):
        #what should be the return value of a path of one node in the graph? (currnet 0)
        path_weight = None
        if path:
            to_nodes = iter(path)
            next(to_nodes)
            edge_weights = [self.get_edge_weight(frm_node,to_node) for frm_node,to_node in zip(path,to_nodes)]
            if None not in edge_weights:
                path_weight = sum(edge_weights)
        return path_weight
  
    def is_reachable(self,frm_name, to_name):
        #what should be the return value of a noe to its self? (current True)
        recheable = False
        if frm_name in self and to_name in self:
            if frm_name not in self._spt:
                self.find_shortest_path(frm_name,to_name)
            predecessor, distance = self._spt[frm_name]
            recheable = to_name in predecessor
            #visited = set()
            #seen = {frm_name}
            #while seen and to_name not in seen:
            #    visiting = seen.pop()
            #    seen.update(node for node in self.neighbors(visiting) if node not in visited)
            #    visited.add(visiting)
            #recheable = to_name in seen
        return recheable
                
    
    def find_shortest_path(self, frm_name, to_name):
        #Dijkstra
        shortest_path = None
        if frm_name in self and to_name in self:
            #Check if the spt was alrady calculated if not calcualte
            if frm_name not in self._spt:
                pq = []
                predecessor = {}
                distance = {frm_name:0}
                visited = set()
                heapq.heappush(pq,(0,frm_name))
                while pq :
                    visiting_distance, visiting_node = heapq.heappop(pq)
                    if visiting_node not in visited:
                        for peer, weight in self.edges_from(visiting_node):
                            if peer not in distance or distance[peer] > visiting_distance + weight:
                                #Relaxation
                                distance[peer] = visiting_distance + weight
                                predecessor[peer] = visiting_node
                                heapq.heappush(pq,(distance[peer],peer))
                        visited.add(visiting_node)
                self._spt[frm_name]=(predecessor,distance)
            
            #Create the path using the SPT
            predecessor, distance = self._spt[frm_name]
            if to_name in predecessor:
                #Creating the path
                shortest_path = [to_name]
                next_hop = to_name
                while next_hop != frm_name:
                    next_hop = predecessor[next_hop]
                    shortest_path.append(next_hop)
                shortest_path.reverse()
        return shortest_path
        
        
     

## Task 2 - Examplary Usage

### Question 1

In [7]:
odd_graph = Graph("odd", [node for node in nodes if node.name() in range(2,10,2)])
even_graph = Graph("even",[node for node in nodes if node.name() in range(1,10,2)])
ten_graph = Graph("ten",[node for node in nodes if node.name()==10])
sample_graph = odd_graph + even_graph + ten_graph
print(sample_graph)

Graph: odd_even_ten
Nodes: 10
  Name: 6 Neighbors: 0
  Name: 4 Neighbors: 1
  Name: 2 Neighbors: 2
  Name: 8 Neighbors: 3
  Name: 5 Neighbors: 1
  Name: 7 Neighbors: 1
  Name: 3 Neighbors: 2
  Name: 9 Neighbors: 3
  Name: 1 Neighbors: 5
  Name: 10 Neighbors: 2
Edges: 20
  Edge: (4,5) Weight: 10
  Edge: (2,3) Weight: 5
  Edge: (2,4) Weight: 10
  Edge: (8,1) Weight: 5
  Edge: (8,2) Weight: 20
  Edge: (8,7) Weight: 5
  Edge: (5,6) Weight: 5
  Edge: (7,6) Weight: 10
  Edge: (3,2) Weight: 15
  Edge: (3,4) Weight: 5
  Edge: (9,2) Weight: 15
  Edge: (9,8) Weight: 20
  Edge: (9,10) Weight: 10
  Edge: (1,2) Weight: 10
  Edge: (1,4) Weight: 20
  Edge: (1,5) Weight: 20
  Edge: (1,6) Weight: 5
  Edge: (1,7) Weight: 15
  Edge: (10,2) Weight: 5
  Edge: (10,3) Weight: 15



### Question 2

### Question 3

In [8]:
nodes.sort(key=lambda u: len([v for v in sample_graph if sample_graph.is_reachable(u.name(),v)]))
print([node.name() for node in nodes])

[6, 5, 7, 4, 2, 3, 10, 1, 8, 9]


### Question 4

In [9]:
def max_weigth_path(graph):
    try:
        max_pair=max(((u,v) for u in graph for v in graph if u!=v and graph.is_reachable(u,v)), \
                    key= lambda e: graph.get_path_weight(graph.find_shortest_path(e[0],e[1])))
        
        max_pair_path = graph.find_shortest_path(max_pair[0],max_pair[1])
        max_pair_path_weight = graph.get_path_weight(max_pair_path)
    except:
        max_pair,max_pair_path,max_pair_path_weight = None, None, None
    return(max_pair,max_pair_path,max_pair_path_weight)
    

max_pair,max_pair_path,max_pair_path_weight = max_weigth_path(sample_graph)
print("The pair with the maximun weigth shortest path is {}".format(max_pair))
print("The path between them is {}".format(max_pair_path))
print("The weight of the paht is {}".format(max_pair_path_weight))

The pair with the maximun weigth shortest path is (9, 5)
The path between them is [9, 2, 4, 5]
The weight of the paht is 35


## Task 3

### Question 1

In [10]:
locations=("Center","North","South","East","West")

def analyze_travels(file_name, locations, date_format):
    total_time = collections.defaultdict(float)
    travels_done = collections.defaultdict(float)
    _from, _start, _to, _stop = 0,1,2,3
    with open(file_name) as f:
        next(f)
        for line in f:
            try:
                rec = line.strip("\n").split(",")
                if rec[_from] in locations and rec[_to] in locations:
                    travel = (rec[_from],rec[_to])
                    duration= (datetime.datetime.strptime(rec[_stop],date_format)-\
                                     datetime.datetime.strptime(rec[_start],date_format)).total_seconds()
                    total_time[travel] += duration
                    travels_done[travel] +=1
            except:
                continue
                
    graph = Graph(file_name)
    for u,v in travels_done:
        graph.build_edge(u,v,total_time[(u,v)]/travels_done[(u,v)])
    return graph
    


travels_EW = analyze_travels("travelsEW.csv",locations, "%d/%m/%Y %Hh%Mm")
travels_WE = analyze_travels("travelsWE.csv",locations,"%I:%M:%S%p ; %b %d %y")


travels = travels_EW + travels_WE

print(travels)

Graph: travelsEW.csv_travelsWE.csv
Nodes: 5
  Name: Center Neighbors: 4
  Name: West Neighbors: 2
  Name: East Neighbors: 1
  Name: South Neighbors: 1
  Name: North Neighbors: 1
Edges: 9
  Edge: (Center,West) Weight: 5377.922848664688
  Edge: (Center,North) Weight: 5308.025078369906
  Edge: (Center,South) Weight: 10806.181818181818
  Edge: (Center,East) Weight: 3512.795031055901
  Edge: (West,North) Weight: 3565.125
  Edge: (West,Center) Weight: 8953.012048192772
  Edge: (East,South) Weight: 3596.98224852071
  Edge: (South,East) Weight: 3582.2093023255816
  Edge: (North,Center) Weight: 3542.7906976744184



### Question 2

In [11]:
max_pair,max_pair_path,max_pair_path_weight = max_weigth_path(travels)
print("The pair with the maximun weigth shortest path is {}".format(max_pair))
print("The path between them is {}".format(max_pair_path))
print("The weight of the paht is {}".format(max_pair_path_weight))

The pair with the maximun weigth shortest path is ('West', 'South')
The path between them is ['West', 'North', 'Center', 'East', 'South']
The weight of the paht is 14217.692977251028


# Part III - Non-Directional Graph

## Task 1

In [86]:
class NonDirectionalGraph(Graph):
    
    def update(self, node):
        super().update(node)
        node = self[node.name()]
        for peer in self._nodes.values():
            if peer in node:
                peer[node] = node[peer]
                #node[peer] = peer[node]
            if node in peer:
                node[peer] = peer[node]
                
    def add_edge(self, frm_name, to_name, weigth):
        if frm_name in self and to_name in self:
            super().add_edge(frm_name,to_name,weigth)
            self[to_name][frm_name] = weigth
    
    def remove_edge(self,frm_name, to_name):
        if frm_name in self and to_name in self:
            super().remove_edge(frm_name,to_name)
            self[to_name].remove_neighbor(frm_name)
            
    def __add__(self, other):
        new_graph = super().__add__(other)
        new_non_directional_graph = NonDirectionalGraph(new_graph._name,[])
        new_non_directional_graph._nodes = new_graph._nodes
        return new_non_directional_graph
    
    def __str__(self):
        msg="Graph: {}\nNodes: {}\n".format(self._name, len(self))
        msg += ''.join("  Name: {} Neighbors: {}\n".format(node,len(list(self.neighbors(node)))) for node in self._nodes)
        edges = ''.join("  Edge: ({},{}) Weight: {}\n".format(frm_n,to_n,self[frm_n][to_n]) for frm_n, to_n in \
                        itertools.combinations(self,2) if  to_n in self[frm_n])
        msg += "Non Directional Edges: {}\n".format(edges.count("Edge: "))
        msg += edges
        return msg
    

## Task 2

In [87]:

def analyze_social(file_name):
    friendships = set()
    _friendA, _friendB, _action = 0,2,3
    with open(file_name) as f:
        for line in f:
            rec = line.strip("\n").split(" ")
            try:
                friends = frozenset([rec[_friendA],rec[_friendB]])
                if rec[_action]=="became":
                    friendships.add(friends)
                if rec[_action]=="cancelled":
                    friendships.discard(friends)
            except:
                continue
    graph = NonDirectionalGraph(file_name)
    for u,v in friendships:
        graph.build_edge(u,v,1)
    return graph

social = analyze_social("social.txt")
print(social)


Graph: social.txt
Nodes: 14
  Name: Simeon Neighbors: 8
  Name: Naphtali Neighbors: 8
  Name: Benjamin Neighbors: 4
  Name: Dan Neighbors: 10
  Name: Manasseh Neighbors: 9
  Name: Judah Neighbors: 9
  Name: Joseph Neighbors: 6
  Name: Zebulun Neighbors: 9
  Name: Asher Neighbors: 7
  Name: Issachar Neighbors: 6
  Name: Gad Neighbors: 6
  Name: Levi Neighbors: 8
  Name: Ephraim Neighbors: 6
  Name: Reuben Neighbors: 4
Non Directional Edges: 50
  Edge: (Simeon,Naphtali) Weight: 1
  Edge: (Simeon,Benjamin) Weight: 1
  Edge: (Simeon,Dan) Weight: 1
  Edge: (Simeon,Manasseh) Weight: 1
  Edge: (Simeon,Judah) Weight: 1
  Edge: (Simeon,Joseph) Weight: 1
  Edge: (Simeon,Zebulun) Weight: 1
  Edge: (Simeon,Ephraim) Weight: 1
  Edge: (Naphtali,Dan) Weight: 1
  Edge: (Naphtali,Manasseh) Weight: 1
  Edge: (Naphtali,Judah) Weight: 1
  Edge: (Naphtali,Issachar) Weight: 1
  Edge: (Naphtali,Gad) Weight: 1
  Edge: (Naphtali,Levi) Weight: 1
  Edge: (Naphtali,Ephraim) Weight: 1
  Edge: (Benjamin,Dan) Weight

### Question 1

In [84]:
def is_clique(graph,group):
    group = set(group)
    return all(node in graph for node in group) and \
            all(set(graph.neighbors(node)) & group == group -{node} for node in group)

all_cliques =list(group for group in \
              itertools.chain.from_iterable(itertools.combinations(social,k) for k in range(len(social)+1))\
              if is_clique(social, group))


len(max(all_cliques,key=len))


5

### Question 2

In [80]:
len(max((clique for clique in all_cliques if "Reuben" in clique),key=len))

3

### Question 3

In [81]:
_,_,max_pair_path_weight = max_weigth_path(social)
print("Max degree of separation is {}".format(max_pair_path_weight))

Max degree of separation is 2


### Question 4

In [15]:
def suggest_friend(graph, u):
    return max((v for v in graph if v != u and v not in graph.neighbors(u)),\
               key= lambda n : len(set(graph.neighbors(n)) & set(graph.neighbors(u))))

suggest_friend(social,"Ephraim")

'Judah'